In [1]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

# 1. gen_mvc (this code)
# 2. gen_view

#------------------------------
#-- 0. Read in 
#------------------------------
assets_df = pd.read_csv('D1_assets.csv')
strats_df = pd.read_csv('S01_Price_Pct.csv')

PRICE_COL = '"Adj Close"' # SQL needs double quotes on field name w space

q = """SELECT 
      drv.PID
     ,drv.AID
     ,drv.Date
     ,drv.qty_chg
     ,a.{price_col}
    FROM strats_df drv
    LEFT JOIN assets_df a on a.AID=drv.AID AND a.Date=drv.Date
    ORDER BY drv.PID, drv.AID, drv.Date
    ;""".format(price_col = PRICE_COL)

df = pysqldf(q)

#----------------------------------------
#-- 2. For each PID AID pair, gen mv cost
#----------------------------------------
# groupby PID, AID
# ref: https://stackoverflow.com/questions/30244952/python-pandas-create-new-column-with-groupby-sum/30244979
# ref code: df['Data4'] = df['Data3'].groupby(df['Date']).transform('sum')
# old code:
# - df[mv_col] = df[qty_chg_col].cumsum() * df[price_col]
# - df[cost_col] = (df[qty_chg_col]*df[price_col]).cumsum()

## market value
df['cum_qty'] = df.groupby(['PID','AID']).cumsum()['qty_chg']
df['mv'] = df['cum_qty'] * df['Adj Close']

## cost
df['cost_chg'] = df['qty_chg']*df['Adj Close']
df['cost'] = df.groupby(['PID','AID']).cumsum()['cost_chg']

#------------------------------
#-- 3. Output to csv
#------------------------------
# Key: Date
df.to_csv('M01 MVC Price Pct.csv',index=False) 

In [3]:
df.head(20)

,PID,AID,Date,qty_chg,Adj Close,cum_qty,mv,cost_chg,cost
0,S01,BABA,2014-09-19,0.000000,93.889999,0.000000,0.000000,0.0,0.0
1,S01,BABA,2014-09-22,0.000000,89.889999,0.000000,0.000000,0.0,0.0
2,S01,BABA,2014-09-23,0.000000,87.169998,0.000000,0.000000,0.0,0.0
3,S01,BABA,2014-09-24,0.000000,90.570000,0.000000,0.000000,0.0,0.0
4,S01,BABA,2014-09-25,0.000000,88.919998,0.000000,0.000000,0.0,0.0
5,S01,BABA,2014-09-26,0.000000,90.459999,0.000000,0.000000,0.0,0.0
6,S01,BABA,2014-09-29,0.000000,88.750000,0.000000,0.000000,0.0,0.0
7,S01,BABA,2014-09-30,0.000000,88.849998,0.000000,0.000000,0.0,0.0
8,S01,BABA,2014-10-01,11.614402,86.099998,11.614402,1000.000000,1000.0,1000.0
9,S01,BABA,2014-10-02,0.000000,87.059998,11.614402,1011.149826,0.0,1000.0
